# Computation on Arrays: Broadcasting
Recall that for arrays of the same size, binary operations are performed on an element-by-element basis:

In [ ]:
a = np.array([0, 1, 2])
b = np.array([5, 5, 5])
a + b

array([5, 6, 7])

Broadcasting allows these types of binary operations to be performed on arrays of different sizes–for example, we can just as easily add a scalar (think of it as a zero-dimensional array) to an array:

In [ ]:
a + 5

array([5, 6, 7])

We can think of this as an operation that stretches or duplicates the value `5` into the array `[5, 5, 5]`, and adds the results. The advantage of NumPy's broadcasting is that this duplication of values does not actually take place, but it is a useful mental model as we think about broadcasting.

We can similarly extend this to arrays of higher dimension. Observe the result when we add a one-dimensional array to a two-dimensional array:

In [ ]:
M = np.ones((3, 3))
M

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [ ]:
M + a

array([[1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.]])

## Rules of Broadcasting
Broadcasting in NumPy follows a strict set of rules to determine the interaction between the two arrays:

1. If the two arrays differ in their number of dimensions, the shape of the one with fewer dimensions is padded with ones on its leading (left) side.
2. If the shape of the two arrays does not match in any dimension, the array with shape equal to 1 in that dimension is stretched to match the other shape.
3. If in any dimension the sizes disagree and neither is equal to 1, an error is raised.

![alt text](https://jakevdp.github.io/PythonDataScienceHandbook/figures/02.05-broadcasting.png)

The light boxes represent the broadcasted values: again, this extra memory is not actually allocated in the course of the operation.

### Broadcasting example 1 

In [ ]:
M = np.ones((2, 3))
a = np.arange(3)

Let's consider an operation on these two arrays. The shape of the arrays are

- ``M.shape = (2, 3)``
- ``a.shape = (3,)``

We see by rule 1 that the array ``a`` has fewer dimensions, so we pad it on the left with ones:

- ``M.shape -> (2, 3)``
- ``a.shape -> (1, 3)``

By rule 2, we now see that the first dimension disagrees, so we stretch this dimension to match:

- ``M.shape -> (2, 3)``
- ``a.shape -> (2, 3)``

The shapes match, and we see that the final shape will be ``(2, 3)``:

In [ ]:
M + a

array([[1., 2., 3.],
       [1., 2., 3.]])

### Broadcasting example 2

Let's take a look at an example where both arrays need to be broadcast:

In [ ]:
a = np.arange(3).reshape((3, 1))
b = np.arange(3)

Again, we'll start by writing out the shape of the arrays:

- ``a.shape = (3, 1)``
- ``b.shape = (3,)``

Rule 1 says we must pad the shape of ``b`` with ones:

- ``a.shape -> (3, 1)``
- ``b.shape -> (1, 3)``

And rule 2 tells us that we upgrade each of these ones to match the corresponding size of the other array:

- ``a.shape -> (3, 3)``
- ``b.shape -> (3, 3)``

Because the result matches, these shapes are compatible. We can see this here:

In [ ]:
a + b

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4]])

### Broadcasting example 3

Now let's take a look at an example in which the two arrays are not compatible:

In [ ]:
M = np.ones((3, 2))
a = np.arange(3)

This is just a slightly different situation than in the first example: the matrix ``M`` is transposed.
How does this affect the calculation? The shape of the arrays are

- ``M.shape = (3, 2)``
- ``a.shape = (3,)``

Again, rule 1 tells us that we must pad the shape of ``a`` with ones:

- ``M.shape -> (3, 2)``
- ``a.shape -> (1, 3)``

By rule 2, the first dimension of ``a`` is stretched to match that of ``M``:

- ``M.shape -> (3, 2)``
- ``a.shape -> (3, 3)``

Now we hit rule 3–the final shapes do not match, so these two arrays are incompatible, as we can observe by attempting this operation:

In [ ]:
M + a

ValueError: ignored

Note that while we've been focusing on the `+` operator here, these broadcasting rules apply to any binary `ufunc`.

### Practical Example: Centering an Array

We saw that ufuncs allow a NumPy user to remove the need to explicitly write slow Python loops. Broadcasting extends this ability.
One commonly seen example is when centering an array of data.
Imagine you have an array of 10 observations, each of which consists of 3 values. We'll store this in a $10 \times 3$ array:

In [ ]:
X = np.random.random((10, 3))
print(X)

[[0.55020314 0.52729805 0.62799153]
 [0.1231663  0.00813549 0.94685107]
 [0.26555134 0.05627657 0.39974775]
 [0.36015553 0.35702367 0.0597209 ]
 [0.06636712 0.4200884  0.32772178]
 [0.99522354 0.98379871 0.72111325]
 [0.12932435 0.26879555 0.91658087]
 [0.27634649 0.29426451 0.16682932]
 [0.72186021 0.26448811 0.37322927]
 [0.42366379 0.30481164 0.28052641]]


We can compute the mean of each feature using the ``mean`` aggregate across the first dimension:

In [ ]:
Xmean = X.mean(axis=0)
Xmean

array([0.39118618, 0.34849807, 0.48203122])

And now we can center the ``X`` array by subtracting the mean (this is a broadcasting operation):

In [ ]:
X_centered = X - Xmean

To double-check that we've done this correctly, we can check that the centered array has near zero mean:

In [ ]:
X_centered.mean(axis=0)

array([-2.22044605e-17,  0.00000000e+00,  3.88578059e-17])

To within machine precision, the mean is now zero.

# Comparisons, Masks, and Boolean Logic
This section covers the use of Boolean masks to examine and manipulate values within NumPy arrays. Masking comes up when you want to extract, modify, count, or otherwise manipulate values in an array based on some criterion: for example, you might wish to count all values greater than a certain value, or perhaps remove all outliers that are above some threshold. In NumPy, Boolean masking is often the most efficient way to accomplish these types of tasks.

## Comparison Operators as ufuncs

We have already introduced ufuncs, and focused in particular on arithmetic operators. We saw that using +, -, *, /, and others on arrays leads to element-wise operations. NumPy also implements comparison operators such as < (less than) and > (greater than) as element-wise ufuncs. The result of these comparison operators is always an array with a Boolean data type. All six of the standard comparison operations are available:

In [ ]:
x = np.array([[1, 2, 3, 4, 5], 
              [4, 5, 6, 5, 4]])

In [ ]:
x < 3  # less than

array([[ True,  True, False, False, False],
       [False, False, False, False, False]])

In [ ]:
x > 3  # greater than

array([[False, False, False,  True,  True],
       [ True,  True,  True,  True,  True]])

In [ ]:
x <= 3  # less than or equal

array([[ True,  True,  True, False, False],
       [False, False, False, False, False]])

In [ ]:
x >= 3  # less than or equal

array([[False, False,  True,  True,  True],
       [ True,  True,  True,  True,  True]])

In [ ]:
x != 3  # not equal

array([[ True,  True, False,  True,  True],
       [ True,  True,  True,  True,  True]])

In [ ]:
x == 3  # not equal

array([[False, False,  True, False, False],
       [False, False, False, False, False]])

In [ ]:
(x == 3) * 1  # not equal

array([[0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0]])

It is also possible to do an element-wise comparison of two arrays, and to include compound expressions:

In [ ]:
(2 * x) == (x ** 2)

array([[False,  True, False, False, False],
       [False, False, False, False, False]])

As in the case of arithmetic operators, the comparison operators are implemented as ufuncs in NumPy; for example, when you write ``x < 3``, internally NumPy uses ``np.less(x, 3)``.
    A summary of the comparison operators and their equivalent ufunc is shown here:

| Operator	    | Equivalent ufunc    || Operator	   | Equivalent ufunc    |
|---------------|---------------------||---------------|---------------------|
|``==``         |``np.equal``         ||``!=``         |``np.not_equal``     |
|``<``          |``np.less``          ||``<=``         |``np.less_equal``    |
|``>``          |``np.greater``       ||``>=``         |``np.greater_equal`` |

## Working with Boolean Arrays
Given a Boolean array, there are a host of useful operations you can do. We'll work with x, the two-dimensional array we created earlier.

### Counting entries
To count the number of True entries in a Boolean array, np.count_nonzero is useful:

In [ ]:
# how many values less than 6?
print(x)
np.count_nonzero(x < 6)

[[1 2 3 4 5]
 [4 5 6 5 4]]
[[ True  True  True  True  True]
 [ True  True False  True  True]]


9

We see that there are eight array entries that are less than 6. Another way to get at this information is to use np.sum; in this case, False is interpreted as 0, and True is interpreted as 1:

In [ ]:
np.sum(x < 6)

9


The benefit of sum() is that like with other NumPy aggregation functions, this summation can be done along rows or columns as well:

In [ ]:
print(x)
np.sum(x < 6, axis = 0) # number of element in each row that are less than 6

[[1 2 3 4 5]
 [4 5 6 5 4]]


array([2, 2, 1, 2, 2])

If we're interested in quickly checking whether any or all the values are true, we can use (you guessed it) np.any or np.all:

In [ ]:
print(np.any(x > 8)) # are there any values greater than 8?
print(np.any(x < 0)) # are there any values less than zero?
print(np.all(x < 10)) # are all values less than 10?
print(np.all(x == 6)) # are all values equal to 6?
print(np.all(x < 8, axis=1)) # are all values in each row less than 8?

False
False
True
False
[ True  True]
[[1 2 3 4 5]
 [4 5 6 5 4]]


In [ ]:
x = np.array([[1, 2, 3, 4, 5], 
              [4, 5, 6, np.nan, 4]])

np.isnan(x)

array([[False, False, False, False, False],
       [False, False, False,  True, False]])

## Boolean operators

We can use bitwise logic operators, `&`, `|`, `^`, and `~`, in order to filter/get useful information from the data. Like with the standard arithmetic operators, NumPy overloads these as ufuncs which work element-wise on (usually Boolean) arrays. 

Combining comparison operators and Boolean operators on arrays can lead to a wide range of efficient logical operations.

The following table summarizes the bitwise Boolean operators and their equivalent ufuncs:

| Operator	    | Equivalent ufunc    || Operator	    | Equivalent ufunc    |
|---------------|---------------------||---------------|---------------------|
|``&``          |``np.bitwise_and``   ||&#124;         |``np.bitwise_or``    |
|``^``          |``np.bitwise_xor``   ||``~``          |``np.bitwise_not``   |

### Boolean Arrays as Masks
In the preceding section we looked at aggregates computed directly on Boolean arrays. A more powerful pattern is to use Boolean arrays as masks, to select particular subsets of the data themselves. Returning to our x array from before, suppose we want an array of all values in the array that are less than, say, 5:

In [ ]:
x = np.array([[1, 2, 3, 4, 5], 
              [4, 5, 6, 5, 4]])

In [ ]:
x < 5

array([[ True,  True,  True,  True, False],
       [ True, False, False, False,  True]])

Now to select these values from the array, we can simply index on this Boolean array; this is known as a masking operation:

In [ ]:
# x[x < 5]
x[(x < 5) | (x == 6)]

array([1, 2, 3, 4, 4, 6, 4])

In [ ]:
print(x < 5)
print(x == 4)
print((x < 5) ^ (x == 6))

[[ True  True  True  True False]
 [ True False False False  True]]
[[False False False  True False]
 [ True False False False  True]]
[[ True  True  True  True False]
 [ True False  True False  True]]


In [ ]:
~(x > 2)

array([[ True,  True, False, False, False],
       [False, False, False, False, False]])

What is returned is a one-dimensional array filled with all the values that meet this condition; in other words, all the values in positions at which the mask array is True.

## Aside: Using the Keywords and/or Versus the Operators &/|

One common point of confusion is the difference between the keywords ``and`` and ``or`` on one hand, and the operators ``&`` and ``|`` on the other hand.
When would you use one versus the other?

The difference is this: ``and`` and ``or`` gauge the truth or falsehood of *entire object*, while ``&`` and ``|`` refer to *bits within each object*.

When you use ``and`` or ``or``, it's equivalent to asking Python to treat the object as a single Boolean entity.
In Python, all nonzero integers will evaluate as True. Thus:

In [ ]:
bool(42), bool(0)

(True, False)

In [ ]:
bool(42 and 0)

False

In [ ]:
bool(42 or 0)

True

When you use ``&`` and ``|`` on integers, the expression operates on the bits of the element, applying the *and* or the *or* to the individual bits making up the number:

In [ ]:
bin(42)

'0b101010'

In [ ]:
bin(59)

In [ ]:
bin(42 & 59)

42

In [ ]:
bin(42 | 59)

59

In [ ]:
bin(-7)

'-0b111'

Notice that the corresponding bits of the binary representation are compared in order to yield the result.

When you have an array of Boolean values in NumPy, this can be thought of as a string of bits where ``1 = True`` and ``0 = False``, and the result of ``&`` and ``|`` operates similarly to above:

In [ ]:
A = np.array([1, 0, 1, 0, 1, 0], dtype=bool)
B = np.array([1, 1, 1, 0, 1, 1], dtype=bool)
A | B

array([ True,  True,  True, False,  True,  True])

Using ``or`` on these arrays will try to evaluate the truth or falsehood of the entire array object, which is not a well-defined value:

In [ ]:
A or B

ValueError: ignored

Similarly, when doing a Boolean expression on a given array, you should use ``|`` or ``&`` rather than ``or`` or ``and``:

In [ ]:
x = np.arange(10)
(x > 4) & (x < 8)

array([False, False, False, False, False,  True,  True,  True, False,
       False])

Trying to evaluate the truth or falsehood of the entire array will give the same ``ValueError`` we saw previously:

In [ ]:
(x > 4) and (x < 8)

ValueError: ignored

So remember this: ``and`` and ``or`` perform a single Boolean evaluation on an entire object, while ``&`` and ``|`` perform multiple Boolean evaluations on the content (the individual bits or bytes) of an object.
For Boolean NumPy arrays, the latter is nearly always the desired operation.

# Fancy Indexing
Fancy indexing is conceptually simple: it means passing an array of indices to access multiple array elements at once. For example, consider the following array:

In [ ]:
rand = np.random.RandomState(42)

x = rand.randint(100, size=10)
print(x)

[51 92 14 71 60 20 82 86 74 74]


In [ ]:
# to access elements with 3, 7, 2 indexes we write
[x[3], x[7], x[2]]

[71, 86, 14]

In [ ]:
index = [3, 7, 2]
x[index]

array([71, 86, 14])

When using fancy indexing, the shape of the result reflects the shape of the index arrays rather than the shape of the array being indexed:

In [ ]:
ind = np.array([[3, 7],
                [4, 5]])
print(x[ind])
print(x)

[[71 86]
 [60 20]]
[51 92 14 71 60 20 82 86 74 74]


#### Indexing with an array of integers

In [ ]:
a = np.arange(0, 100, 10)
a

array([ 0, 10, 20, 30, 40, 50, 60, 70, 80, 90])

Indexing can be done with an array of integers, where the same index is repeated several time.

In [ ]:
# note: [2, 3, 2, 4, 2] is a Python list

a[[2, 3, 2, 4, 2]] 

array([20, 30, 20, 40, 20])

New values can be assigned with this kind of indexing.

In [ ]:
a[[9, 7]] = -100
a

array([   0,   10,   20,   30,   40,   50,   60, -100,   80, -100])

This works for multidimentional arrays as well:

In [ ]:
X = np.arange(12).reshape((3, 4))
X

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

Like with standard indexing, the first index refers to the row, and the second to the column:

In [ ]:
row = np.array([0, 1, 2])
col = np.array([2, 1, 3])
X[row, col]

array([ 2,  5, 11])

## Combined Indexing
For even more powerful operations, fancy indexing can be combined with the other indexing schemes we've seen:

In [ ]:
print(X)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


In [ ]:
# combine fancy and simple indices
X[2, [2, 0, 1]]

array([10,  8,  9])

In [ ]:
# combine fancy indexing with slicing
X[1:, [2, 0, 1]]

array([[ 6,  4,  5],
       [10,  8,  9]])

In [ ]:
# combine fancy indexing with masking
mask = np.array([1, 0, 1, 0], dtype=bool)
X[row[:, np.newaxis], mask]

array([[ 0,  2],
       [ 4,  6],
       [ 8, 10]])

In [ ]:
X[0]

array([0, 1, 2, 3])

# Sorting Arrays

There are built-in NumPy methods that sort arrays efficiently:
`np.sort` and `np.argsort`. 

Although Python has built-in sort and sorted functions to work with lists, we won't discuss them here because NumPy's np.sort function turns out to be much more efficient and useful for our purposes. By default np.sort uses an $O(n \log n)$, quicksort algorithm, though mergesort and heapsort are also available.


In [ ]:
def quicksort(arr):
  if len(arr) <= 1:
      return arr
  pivot = arr[len(arr) // 2]
  left = [x for x in arr if x < pivot]
  middle = [x for x in arr if x == pivot]
  right = [x for x in arr if x > pivot]
  return quicksort(left) + middle + quicksort(right)

In [ ]:
big_array = np.random.rand(1000000)
%timeit sorted(big_array)
%timeit np.sort(big_array)
%timeit quicksort(big_array)

1 loop, best of 3: 1.07 s per loop
10 loops, best of 3: 131 ms per loop
1 loop, best of 3: 6.47 s per loop


In [ ]:
x = np.array([2, 1, 4, 3, 5])
# quicksort(x)
i = np.argsort(x)
print(i)

These indices can then be used (via fancy indexing) to construct the sorted array if desired:

In [ ]:
x[i]

## Sorting along rows or columns
A useful feature of NumPy's sorting algorithms is the ability to sort along specific rows or columns of a multidimensional array using the axis argument.

In [ ]:
rand = np.random.RandomState(42)
X = rand.randint(0, 10, (4, 6))
print(X)

[[6 3 7 4 6 9]
 [2 6 7 4 3 7]
 [7 2 5 4 1 7]
 [5 1 4 0 9 5]]


In [ ]:
# sort each column of X
np.sort(X, axis=0)

array([[2, 1, 4, 0, 1, 5],
       [5, 2, 5, 4, 3, 7],
       [6, 3, 7, 4, 6, 7],
       [7, 6, 7, 4, 9, 9]])

In [ ]:
# sort each row of X
np.sort(X, axis=1)

array([[3, 4, 6, 6, 7, 9],
       [2, 3, 4, 6, 7, 7],
       [1, 2, 4, 5, 7, 7],
       [0, 1, 4, 5, 5, 9]])

# Numpy for Linear Algebra

By now we are already familiar with a range concepts of linear algebra - the branch of mathematics concerned with vector spaces and the mappings between those spaces. **NumPy** has a package called **linalg** which includes functions for implementing standard linear algebra algorithms. Those functions rely on libraries that provide efficient low level implementations of standard linear algebra algorithms. Those libraries are
 * BLAS (Basic Linear Algebra Subprograms) - software library written in Fortran that provides low-level routines for performing common linear algebra operations such as vector addition, scalar multiplication, dot products, linear combinations, and matrix multiplication.
 * LAPACK (Linear Algebra Package) - software library written in *Fortran 90* that provides routines for solving systems of linear equations and linear least squares, eigenvalue problems, and matrix decomposition.
 The above mentioned libraries provide efficient low level implementations 

## Matrix and vector products

In [ ]:
# similarly we can construct matrices
# we can multiply matrices with
a = np.array([
              [2, 3, 1],
              [3, -1, 0]  # 2 x 3
]) 

b = np.array([
              [1, -3],
              [4, 0],
              [2, -2]   # 3 x 2
])   

print(a.shape)
print(b.shape)

(2, 3)
(3, 2)


## Transposing Matrices

In [ ]:
# get the transpose of the matrix
print('First method', np.transpose(b), sep='\n', end='\n\n')
print('Second method', b.T, sep='\n')
print('Third method', b.transpose(), sep='\n', end='\n\n')

First method
[[ 1  4  2]
 [-3  0 -2]]

Second method
[[ 1  4  2]
 [-3  0 -2]]
Third method
[[ 1  4  2]
 [-3  0 -2]]



## Inverting Matrices

In [ ]:
d = np.array([[3, 2, 3],
              [1, 3, 4],
              [3, 4, 5]]) # 3 x 3

# get the inverse matrix
d_inv = np.linalg.inv(d)
print('Inverse Matrix', d_inv, sep='\n', end='\n\n')
print("Checking the inverse d' * d", np.around(d_inv @ d, 2),
      sep='\n', end='\n\n')
print("Checking the inverse d * d'", abs(np.around(d @ d_inv, 2)), sep='\n')

Inverse Matrix
[[ 0.25 -0.5   0.25]
 [-1.75 -1.5   2.25]
 [ 1.25  1.5  -1.75]]

Checking the inverse d' * d
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Checking the inverse d * d'
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [ ]:
# transpose of a transpose is the initial matrix
print(b, end='\n\n')
print(b.T.T)

[[ 1 -3]
 [ 4  0]
 [ 2 -2]]

[[ 1 -3]
 [ 4  0]
 [ 2 -2]]


In [ ]:
# we can multiply matrices using the following commands
# 2 x 3 * 3 x 2 = 2 x 2
print('First method', a.dot(b), sep='\n', end='\n\n')
print('Second method', np.matmul(a, b), sep='\n', end='\n\n')
print('The shortest method', a @ b, sep='\n')

First method
[[16 -8]
 [-1 -9]]

Second method
[[16 -8]
 [-1 -9]]

The shortest method
[[16 -8]
 [-1 -9]]


In [ ]:
# create an itentity matrix 
identity_matrix = np.eye(3)
print(identity_matrix, end='\n\n')

# checking one of the properties of the identity matrix
print(a @ identity_matrix, end='\n\n')

# checking if the corresponding elements are the same
print(a @ identity_matrix == a, end='\n\n')
  
# checking if the matrices are the same
print('Checking the equality of arrays (method 1):',
      (a @ identity_matrix == a).all())
# or
print('Checking the equality of arrays (method 2):', 
      np.allclose(a @ identity_matrix, a))

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

[[ 2.  3.  1.]
 [ 3. -1.  0.]]

[[ True  True  True]
 [ True  True  True]]

Checking the equality of arrays (method 1): True
Checking the equality of arrays (method 2): True


In [ ]:
# let's check some other properties of matrices
c = np.array([
              [2, 0, 4, 5, -9],  # 2 x 5
              [1, 4, 3, 8, -2]
])

# associativity: 
# for a, b, c matrices with dim(a)=m x n, dim(b)=n x p, dim(c)=p x q
# (a*b)*c=a*(b*c)
# in our case dim(a)=2 x 3, dim(b)=3 x 2, dim(c)=2 x 5
print((a @ b) @ c, end='\n\n')
print(a @ (b @ c))

[[  24  -32   40   16 -128]
 [ -11  -36  -31  -77   27]]

[[  24  -32   40   16 -128]
 [ -11  -36  -31  -77   27]]


In [ ]:
# distributivity: 
# for a, b, c with dim(a)=dim(b)=m x n, dim(c)=n x p 
# in our case dim(a)=2 x 3, dim(a.T)=dim(b)=3 x 2
print((a.T + b) @ c, end='\n\n')
print(a.T @ c + b @ c)

[[  6   0  12  15 -27]
 [ 13  -4  25  27 -61]
 [  4  -8   6  -1 -23]]

[[  6   0  12  15 -27]
 [ 13  -4  25  27 -61]
 [  4  -8   6  -1 -23]]


Now let's compute inner products of given vectors

In [ ]:
# defining column vectors
x = np.array([[1, 2, 3]]).T
y = np.array([[2, 3, 4]]).T

# if the inputs of np.dot are 1d arrays it returns the inner product
# we saw above that for 2d arrays it is the matrix multiplication 
print('Method 1:', np.dot(x.T, y))

# np.vdot computes the dot product between 2 vectors
# if we give multidimensional arrays as inputs it flattens them and 
# computes the dot product, instead of matrix product
print('Method 2:', np.vdot(x, y))

# computing via the formula of dot product
print('Method 3:', x.T @ y)

# matrix multiplication for 1d arrays
print('Method 4:', np.matmul(x.T, y))

# takes column vectors as input
print('Method 5:', np.inner(x.T, y.T))

Method 1: [[20]]
Method 2: 20
Method 3: [[20]]
Method 4: [[20]]
Method 5: [[20]]


In [ ]:
# (a+b)^T=a^T + b^T
print((a.T+b).T, end='\n\n')
print(a + b.T)

[[ 3  7  3]
 [ 0 -1 -2]]

[[ 3  7  3]
 [ 0 -1 -2]]


### Warm up Exercises

NOTE: after you write your code run the cells with **assert** statements for each task in below cell. If an Error is thrown, then you have done something wrong. If nothing is printed then , the exercise is done correctly. Do NOT modify cells with **assert** statements.

Write a function which calculates The following formula. $(XX^{T})^{-1}$

In [ ]:
def transpose_inv(input_x):
  # your code here

In [ ]:
assert transpose_inv(np.array([[1, 0],[1, 5]]))[1][1] == 0.04
assert np.around(transpose_inv(np.array([[10, 0],[20, 55]])),2)[0][0] == 0.01
assert np.around(transpose_inv(np.array([[10, 1],[78, 10]])),2)[0][1] == -1.63

## Norms and other numbers

In [ ]:
# computing the norm norm, trace, rank
x = np.array([5, 4, 3])
print('First Method', np.linalg.norm(x))
print('Second Method', np.sqrt(x @ x.T))

First Method 7.0710678118654755
Second Method 7.0710678118654755


Checking that $||A\boldsymbol{x}||=||\boldsymbol{x}||$, when A is an orthogonal matrix. 


In [ ]:
A = np.array([[2/3, 1/3, 2/3],
              [-2/3, 2/3, 1/3],
              [1/3, 2/3, -2/3]])

print(np.linalg.norm(A @ x))

7.071067811865475


In [ ]:
# compute the determinant of a matrix
print('det(A) =', np.linalg.det(A))

det(A) = -1.0


In [ ]:
# compute the trace of a matrix
# this is not a part of the linalg package
print('Tr(A) = ', np.trace(A))
print('Tr(A) = ', np.sum(np.diag(A)))

Tr(A) =  0.6666666666666666
Tr(A) =  0.6666666666666666


In [ ]:
# compute the rank of a matrix
print('Rank(A) =', np.linalg.matrix_rank(A))

Rank(A) = 3


## Solving equations

Suppose we want to solve $$A\boldsymbol{x}=b$$

In [ ]:
# solving Ax=b linear equation
A = np.array([
              [3, 1],
              [1, 2]
])

b = np.array([9, 8])

x = np.linalg.solve(A, b)
print(x, end='\n\n')
# checking the solution
print(np.allclose(A @ x, b))

[2. 3.]

True


Using the inverse of A
$$A^{-1}A\boldsymbol{x}=A^{-1}b \Rightarrow 
 \boldsymbol{x}=A^{-1}b$$

In [ ]:
print(np.linalg.inv(A) @ b)

[2. 3.]


Suppose $A$ is not a square matrix, we can find an approximate solution to the equation $A\boldsymbol{x}=b$ using the least squares method, finding $\boldsymbol{x}$ that minimizes $||A\boldsymbol{x}-b||^{2}$  

In [ ]:
A = np.array([[2, 0],
              [-1, 1],
              [0, 2]])

b = np.array([1, 0, -1])

print(np.linalg.lstsq(A, b, rcond=None)[0])

[ 0.33333333 -0.33333333]


If $A$ has linearly independent columns (left-invertible), then the vector $$\hat{\boldsymbol{x}}=(A^{T}A)^{-1}A^{T}b=A^{\dagger}b,$$
where $A^{\dagger}$ is the *pseudo-inverse* of a left-invertible matrix.

In [ ]:
print(np.linalg.inv(A.T @ A) @ A.T @ b)
print(np.linalg.pinv(A))

[ 0.33333333 -0.33333333]
[[ 0.41666667 -0.16666667  0.08333333]
 [ 0.08333333  0.16666667  0.41666667]]


## Matrix eigenvalues

In [11]:
# compute the eigenvalues and right eigenvectors 
A = np.array([[2, 0, 3],
              [0, 3, 1],
              [0, 0, 4]])

eigenvalues, eigenvectors = np.linalg.eig(A)

print('Eigenvalues:', eigenvalues)
print('Eigenvectors as columns:\n', eigenvectors)

Eigenvalues: [2. 3. 4.]
Eigenvectors as columns:
 [[1.         0.         0.72760688]
 [0.         1.         0.48507125]
 [0.         0.         0.48507125]]


In [12]:
# compute the eigenvalues only
print(np.linalg.eigvals(A))

[2. 3. 4.]


We know that for $A \in \mathbb{R}^{n \times n}$
$$det(A)=\prod_{i=1}^{n}\lambda_i$$
$$Tr(A)=\sum_{i=1}^{n}\lambda_i$$

In [13]:
# checking the above
print('det(A) = ', np.linalg.det(A))
print('det(A) = ', np.prod(eigenvalues))
print('Tr(A) = ', np.sum(eigenvalues))
print('Tr(A) = ', np.trace(A))

det(A) =  23.999999999999993
det(A) =  24.0
Tr(A) =  9.0
Tr(A) =  9


## Matrix decompositions

### Eigendecomposition

A squared matrix $A \in \mathbb{R}^{n \times n}$ can be factored into 
$$A=PDP^{-1},$$
where $P \in \mathbb{R}^{n \times n}$ consists of the eigenvectors of $A$ and $D$ is a diagonal matrix with eigenvalues of $A$, if and only if the eigenvectors of $A$ form a basis of $\mathbb{R}^{n}$.

Symmetric matrices can always be diagonalized.

In [14]:
print(A)
# D = np.eye(A.shape[0]) * eigenvalues
D = np.diag(eigenvalues)
print(eigenvectors @ D @ np.linalg.inv(eigenvectors))

[[2 0 3]
 [0 3 1]
 [0 0 4]]
[[2. 0. 3.]
 [0. 3. 1.]
 [0. 0. 4.]]


### Cholesky Decomposition

A symmetric, positive definite matrix $A$ can be factorized into a product $$A=LL^{T},$$
where L (*Cholesky factor* of $A$) is a unique lower-triangular matrix with positive diagonal elements.


In [15]:
A = np.array([[2, 1],
              [1, 4]])

L = np.linalg.cholesky(A)
print(L)
print('A = L*L^T:',np.allclose(A, L @ L.T))

[[1.41421356 0.        ]
 [0.70710678 1.87082869]]
A = L*L^T: True


The Cholesky decomposition also allows us to compute determinants
very efficiently. We
know that $$det(A) = det(L) \cdot det(L^{T}) = det(L)^{2}$$. Since L is a triangular matrix, the determinant is simply the product of its diagonal entries 
$$det(A) = \prod_{i=1}l_{ii}^{2}.$$

In [16]:
print('det(A) = ', np.linalg.det(A))
print('det(A) = ', np.prod(np.diag(L) ** 2))

det(A) =  7.000000000000001
det(A) =  7.000000000000002


### Singular Value Decomposition

Let $A \in \mathbb{R}^{m \times n}$ be a rectangular matrix of rank $r \in [0, min(m, n)].$ The SVD of $A$ is a decomposition of the form
$$A=U \Sigma V^{T},$$
where $U \in \mathbb{R}^{m \times m}$ (left-singular vectors) and $V \in \mathbb{R}^{n \times n}$ (right-singular vectors) are orthogonal matrices, $\Sigma \in \mathbb{R}^{m \times n}$ matrix with diagonal elements (singular values) $\sigma_{ii}\geq 0$ and $\Sigma_{ij}=0$ for $i\neq j.$

The singular value decomposition of a matrix is a central matrix
decomposition method in linear algebra, because it can be
applied to all matrices, not only to square matrices, and it **always exists**.

In [17]:
A = np.array([[1, 2, 3], 
              [4, 5, 6]])

u, s, vh = np.linalg.svd(A)
print('u:', u.shape)
print('s:', s.shape)
print('vh:', vh.shape)

u: (2, 2)
s: (2,)
vh: (3, 3)


In [18]:
print(u)
print(s)
print(vh)

[[-0.3863177  -0.92236578]
 [-0.92236578  0.3863177 ]]
[9.508032   0.77286964]
[[-0.42866713 -0.56630692 -0.7039467 ]
 [ 0.80596391  0.11238241 -0.58119908]
 [ 0.40824829 -0.81649658  0.40824829]]


In [19]:
S = np.hstack((np.diag(s), np.zeros((2,1))))
print(S)
print(u @ S @ vh)

[[9.508032   0.         0.        ]
 [0.         0.77286964 0.        ]]
[[1. 2. 3.]
 [4. 5. 6.]]


One possible application of SVD is computing the pseudoinverse that we saw above. Suppose that $$A=U \Sigma V^{T},$$ then 
\begin{equation}
\begin{split}
A^{\dagger} &= (A^TA)^{-1}A^T = \\
&= (V \Sigma^{T} U^T U \Sigma V^{T})^{-1}A^{T}=\\
&=(V \Sigma^{T} \Sigma V^{T})^{-1} V \Sigma^{T} U^T=\\
&= (V \Sigma^{2} V^{T})^{-1} V \Sigma^{T} U^T=\\
&=V \Sigma^{-2} V^T V \Sigma^T U^T =\\
&=V \Sigma^{*} U^{T},
\end{split}
\end{equation}
where $\Sigma^{*}$ is the pseudoinverse of $\Sigma$, which is formed by replacing every non-zero diagonal entry by its reciprocal and transposing the resulting matrix.

In [20]:
A_pinv = vh.T @ np.vstack((np.diag(1/s), np.zeros((1, 2)))) @ u.T
print(A_pinv)
print(np.linalg.pinv(A))

[[-0.94444444  0.44444444]
 [-0.11111111  0.11111111]
 [ 0.72222222 -0.22222222]]
[[-0.94444444  0.44444444]
 [-0.11111111  0.11111111]
 [ 0.72222222 -0.22222222]]
